In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode  # noqa: F401
except ImportError:
    !python -m pip install openmdao[notebooks]

# ImplicitFuncComp

`ImplicitFuncComp` is a component that provides a shortcut for building an ImplicitComponent based on a python function. That function takes inputs and states as arguments and returns residual values. The function may take some inputs that are non-differentiable and are assumed to be static during the computation of derivatives.  These static values may be of any hashable type.  All other arguments and return values must be either floats or numpy arrays. The mapping between a state argument and its residual output must be specified in the metadata when the output (state) is added by setting 'resid' to the name of the residual.

It may seem confusing to use `add_output` to specify state variables since the state variables
are actually input arguments to the function, but in OpenMDAO's view of the world, states are outputs so we use `add_output` to specify them.  Also, using the metadata to specify which input arguments are actually states gives more flexibility in terms of how the function arguments are ordered. For example, if it's desirable for a function to be passable to `scipy.optimize.newton`, then the function's arguments can be ordered with the states first, followed by the inputs, in order to match the order expected by `scipy.optimize.newton`.

The `add_output` function is part of the [Function Metadata API](../func_api.ipynb).  You use this API to specify various metadata that OpenMDAO needs in order to properly configure a fully functional implicit component. You should read and understand the [Function Metadata API](../func_api.ipynb) before you continue with this section.

## ImplicitFuncComp Options


In [ ]:
import openmdao.api as om
def func(a):
    y = a * 2.
    return y
om.show_options_table(om.ImplicitFuncComp(func))

## ImplicitFuncComp Constructor

The call signature for the `ImplicitFuncComp` constructor is:

```{eval-rst}
    .. automethod:: openmdao.components.implicit_func_comp.ImplicitFuncComp.__init__
        :noindex:
```

## ImplicitFuncComp Example: A simple implicit function component

The simplest implicit function component requires the definition of a function that takes
inputs and states as arguments and returns residual values.  This function maps to the `apply_nonlinear`
method in the OpenMDAO component API. Here's an example:

In [ ]:
import openmdao.api as om
import openmdao.func_api as omf

def apply_nl(a, b, c, x):  # inputs a, b, c and state x
    R_x = a * x ** 2 + b * x + c
    return R_x

f = (omf.wrap(apply_nl)
        .add_output('x', resid='R_x', val=0.0)
        .declare_partials(of='*', wrt='*', method='cs')
        )

p = om.Problem()
p.model.add_subsystem('comp', om.ImplicitFuncComp(f))

p.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False, iprint=0)

# need this since comp is implicit and doesn't have a solve_linear
p.model.linear_solver = om.DirectSolver()

p.setup()

p.set_val('comp.a', 2.)
p.set_val('comp.b', -8.)
p.set_val('comp.c', 6.)
p.run_model()


In [ ]:
from openmdao.utils.assert_utils import assert_check_partials, assert_check_totals

assert_check_partials(p.check_partials(includes=['comp'], out_stream=None), atol=1e-5)
assert_check_totals(p.check_totals(of=['comp.x'], wrt=['comp.a', 'comp.b', 'comp.c'], out_stream=None))